<a href="https://colab.research.google.com/github/TeleStats/PA22_replication/blob/main/PA22_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Replication package for for PA2022 submission "Face detection, tracking, and classification from large-scale news archives for analysis of key political figures"

####Clone repository and install requirements

In [ ]:
# Clone the repo from github and position to the main folder
!git clone https://github.com/TeleStats/PA22_replication
%cd PA22_replication
!mkdir figures

####Download face detection, face features, and ground truth annotations

In [ ]:
# Download and prepare data folder
# This contains:
# Download the embeddings corresponding to the individuals' models
# Download precomputed detections and face features embeddedings
# Download ground truth data for evaluation
!wget --no-check-certificate www.satoh-lab.nii.ac.jp/member/agirbau/telestats/files/data.tar.gz
!tar -xf data.tar.gz data
!rm data.tar.gz

##Requirements for replication package
####Install requirements to replicate the results in google colab

In [ ]:
# Install project requirements
!pip install -r requirements_colab.txt
!pip install fiftyone

##Configuration
####Specify the configuration options
######**channel** --> news7-lv (NHK), hodost-lv (HODO Station), CNNW (CNN), FOXNEWSW (FOX), MSNBCW (MSNBC)
######**detector** --> dfsd, mtcnn, yolo
######**classifier** --> knn_3 (KNN), fcg_average_centroid (Centroid), fcg_average_vote (Vote), fcgNT_average_vote (for "No Tracking" in Table 6)



In [ ]:
#### Configuration options ####
channel = "CNNW" # news7-lv (NHK), hodost-lv (HODO Station), CNNW (CNN), FOXNEWSW (FOX), MSNBCW (MSNBC)
detector = "yolo"  # dfsd, mtcnn, yolo
feats = "resnetv1"  # resnetv1 (Inception-resnet as backbone)
classifier = "fcg_average_vote"  # knn_3, fcg_average_centroid, fcg_average_vote, fcgNT_average_vote (for "No Tracking" experiment in Table 6)
models_path = "faces_politicians" if channel in ['news7-lv', 'hodost-lv'] else "faces_us_individuals"

##Run the tracking + classification part of the method
####We already provide detections and face embeddings (previously downloaded). Run tracking and classification part to assign an ID to each detected face.
######Please, run this code for all the different configurations, as done in the paper.
######e.g. channel = "CNNW", detector = "yolo", classifier = "fcg_average_vote"

In [ ]:
# Run classification for the specified options
!python src/face_classifier.py train "$channel" --models_path data/"$models_path" --detector "$detector" --feats "$feats" --mod_feat "$classifier"

##Replication of Tables 3-6
####Method performance with respect to face size

In [ ]:
# Table 3
# Amount of missed detections for the specified detector
!python src/metrics.py train "$channel" --models_path data/"$models_path" --detector "$detector" --use_dets

In [ ]:
# Tables 4, 5, 6
# Run evaluation for for the specified options (detector + classifier)
!python src/metrics.py train "$channel" --models_path data/"$models_path" --detector "$detector" --feats "$feats" --mod_feat "$classifier"

##Replication of Figures 5-6
####Method performance with respect to face size
######Please, before reproducing this experiment, run classification for all channels with the following configuration:
######**detector** = "yolo", **classifier** = "fcg_average_vote"

In [ ]:
# Generate dataset with fiftyone
!python src/convert_dataset_to_fiftyone.py

In [ ]:
# Populate the dataset with the detections/classification of the key individuals
!python src/convert_results_to_fiftyone.py

In [ ]:
import pandas as pd
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

print(fo.list_datasets())

####Run the cells below to replicate the results for figures 5-6 of the specified dataset

In [ ]:
# Specify dataset
dataset_orig = fo.load_dataset(channel)
us_dataset_list = ["CNNW", "FOXNEWSW", "MSNBCW"]
# Do "evaluate_detections" to compute iou to be able to threshold wrt iou for US data evaluation
if dataset_orig.name in us_dataset_list:
    dataset_orig.evaluate_detections("yolo-resnetv1-fcg_average_vote", "ground_truth", eval_key="eval", classwise=False)

# For NHK and hodo station
#years_list = [str(i) for i in range(2013, 2022)]
years_list = [str(i) for i in range(2000, 2022)]
view_analysis = dataset_orig.match(F("year").is_in(years_list))

# For US evaluation
# Filter the detections based on the IoU threshold
if dataset_orig.name in us_dataset_list:
    view_analysis = view_analysis.filter_labels("yolo-resnetv1-fcg_average_vote", F("eval_iou") > 0.001).clone()

# Generate different views depending on the bounding box sizes 
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)
# [very small, small, small-medium, medium, medium-large, large, very large]
# Average bbox for NHK = 78x78, HODO = 52x52. US dataset around 135 x 135.
# Smallest NHK = 3x3, HODO = 2x2. US = 35x35
# Largest NHK = 258x258, HODO = 174x174. US = 390x390

boxes_areas = list(map(int, list(np.asarray([8, 16, 32, 64, 96, 128, 156]) ** 2)))
boxes_filter_list = []

for i in range(len(boxes_areas)):
    if i == 0:
        # First case
        boxes_filter = bbox_area <= boxes_areas[i]
    else:
        # Cases in the middle
        boxes_filter = (bbox_area > boxes_areas[i-1]) & (bbox_area <= boxes_areas[i])

    boxes_filter_list.append(boxes_filter)
        
# Last case
boxes_filter_list.append(bbox_area > boxes_areas[-1])


In [ ]:
# Generate views that contains only the filtered bboxes depending on size
views_list = []

for box_filter in boxes_filter_list:
#for box_filter in [small_boxes, medium_boxes]:
    view_filtered = (
        view_analysis
        .filter_labels("ground_truth", box_filter)
        .filter_labels("yolo-resnetv1-fcg_average_vote", box_filter)
        .filter_labels("yolo-resnetv1-fcg_average_vote", F("label") != "-1")
    )
    views_list.append(view_filtered)

In [ ]:
# Run evaluation for the generated filtered views
results_list = []
if dataset_orig.name in us_dataset_list:
    iou_threshs = [0.4, 0.45, 0.5, 0.55, 0.6]
else:
    iou_threshs = None

for view_filtered in views_list:
    results_filtered = view_filtered.evaluate_detections(
        "yolo-resnetv1-fcg_average_vote",
        gt_field="ground_truth",
        eval_key="eval",
        compute_mAP=True,
        iou_threshs=iou_threshs,  # For US evaluation
    )

    results_list.append(results_filtered)

In [ ]:
rows_df = []
# 186**2 is for visualization purposes, representing [156-]
for res, box_area in zip(results_list, boxes_areas + [186**2]):
    res_map = round((max(res.mAP(), 0) * 100), 1)
    res_f1 = round(res.metrics()['fscore'], 3)
    box_size = int(np.sqrt(box_area))
    rows_df.append([box_area, box_size, res_map, res_f1])
    print(f"mAP: {res_map}, F1: {res_f1}")

df_res = pd.DataFrame(data=rows_df, columns=['area', 'box_size', 'map', 'f1'])
print(df_res)

In [ ]:
import plotly.express as px

# mAP
fig = px.line(df_res, x="box_size", y="map", text="map", title=f"mAP per bounding box size for {dataset_orig.name}")
fig.update_traces(textposition="bottom right")

fig.update_xaxes(
    title="Bounding box size"
)
fig.update_yaxes(
    title="mAP"
)

fig.write_image(f"figures/results_map_face_size_{dataset_orig.name}.pdf")
fig.show()

# F1 score
fig = px.line(df_res, x="box_size", y="f1", text="f1", title=f"F-score per bounding box size for {dataset_orig.name}")
fig.update_traces(textposition="bottom right")

fig.update_xaxes(
    title="Bounding box size"
)
fig.update_yaxes(
    title="F-score"
)

fig.write_image(f"figures/results_f1_face_size_{dataset_orig.name}.pdf")
fig.show()